In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 20 06:20:43 2023

@author: abdullahalbinsaleh

This version uses normal random data splitting technique 
"""

#-------------------------import librarires-------------------------#
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier 
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
#-------------------------Load dataset-------------------------#
moneyFlow = pd.read_excel("moneyFlow.xlsx")

#-------------------------Data Exploration-------------------------#
moneyFlow.head()
moneyFlow.info()
moneyFlow.columns


#-------------------------Data Preparation & Pre-Processing-------------------------#
#Drop missing values 
moneyFlow.dropna(inplace=True)
moneyFlow = moneyFlow.drop('Unnamed: 0', axis=1)
#drop $ sign from Spot feature 
# convert the 'Spot' column to a string
moneyFlow['Spot'] = moneyFlow['Spot'].astype(str)
moneyFlow['Spot'] = moneyFlow['Spot'].str.replace("$", "")
# convert the 'Spot' column back to float
moneyFlow['Spot'] = moneyFlow['Spot'].astype(float)

#convert Date and Expiration to timestamp 
moneyFlow["Date"] = pd.to_datetime(moneyFlow["Date"])
moneyFlow["Expiration"] = pd.to_datetime(moneyFlow["Expiration"])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183504 entries, 0 to 183503
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Unnamed: 0     183504 non-null  object        
 1   Date           183504 non-null  datetime64[ns]
 2   Time           183504 non-null  object        
 3   Ticker         183504 non-null  object        
 4   Expiration     183504 non-null  datetime64[ns]
 5   Strike         183504 non-null  float64       
 6   C/P            183504 non-null  object        
 7   Spot           183504 non-null  float64       
 8   Details        183504 non-null  object        
 9   Type           183504 non-null  object        
 10  Prem           183504 non-null  float64       
 11  Sentiment      183504 non-null  object        
 12  Execution      183504 non-null  object        
 13  Open Interest  183504 non-null  int64         
 14  Volume         183504 non-null  int64         
dtype

C:\Users\JBarr\AppData\Local\Temp\ipykernel_35824\3573196668.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  moneyFlow['Spot'] = moneyFlow['Spot'].str.replace("$", "")


In [2]:

#-------------------------Feature Engineering -------------------------#
#Split Details feature into two features 
moneyFlow[["Size", "Price"]] = moneyFlow["Details"].str.split("@", expand=True)
#drop the Details feature 
moneyFlow.drop(columns=["Details"], inplace=True)

#Find options duration in days 
moneyFlow["Duration"] = (moneyFlow["Expiration"]-moneyFlow["Date"]).dt.days
#drop the Date and Expiration deatuers 
moneyFlow.drop(columns=["Date"], inplace=True)
moneyFlow.drop(columns=["Expiration"], inplace=True)


#Convert categoircal variables into dummy variables 
moneyFlow = pd.get_dummies(moneyFlow, columns=["Type", "Execution", "C/P"])


#-------------------------Feature Selection Using Domain Knowledge -------------------------#
#Drop useless predictors using domain knowledge
moneyFlow.drop(columns=["Time", "Ticker", "Price", "Size", "Volume", "Open Interest"], inplace=True)

#new data set columns 
moneyFlow.info()
moneyFlow.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183504 entries, 0 to 183503
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Strike                 183504 non-null  float64
 1   Spot                   183504 non-null  float64
 2   Prem                   183504 non-null  float64
 3   Sentiment              183504 non-null  object 
 4   Duration               183504 non-null  int64  
 5   Type_SWEEP             183504 non-null  uint8  
 6   Type_TRADE             183504 non-null  uint8  
 7   Execution_ABOVE ASK    183504 non-null  uint8  
 8   Execution_AT ASK       183504 non-null  uint8  
 9   Execution_AT BID       183504 non-null  uint8  
 10  Execution_AT MIDPOINT  183504 non-null  uint8  
 11  Execution_BELOW BID    183504 non-null  uint8  
 12  C/P_CALL               183504 non-null  uint8  
 13  C/P_PUT                183504 non-null  uint8  
dtypes: float64(3), int64(1), object(1), 

Index(['Strike', 'Spot', 'Prem', 'Sentiment', 'Duration', 'Type_SWEEP',
       'Type_TRADE', 'Execution_ABOVE ASK', 'Execution_AT ASK',
       'Execution_AT BID', 'Execution_AT MIDPOINT', 'Execution_BELOW BID',
       'C/P_CALL', 'C/P_PUT'],
      dtype='object')

In [3]:

#-------------------------Cluestring-------------------------#


#clustering based  on Premium and Type of Trade
premType = moneyFlow.iloc[:,[2,5,6]]

#scale the data
scaler = StandardScaler()
premType_scaled = scaler.fit_transform(premType)
premType_scaled = pd.DataFrame(premType_scaled, columns=premType.columns, index= premType.index)


In [4]:
kmodel = KMeans()


In [7]:
pip install --upgrade threadpoolctl



  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0


In [8]:

#2 clusters 
premType_2_Clusters = KMeans(n_clusters=2,random_state=0).fit(premType_scaled)


c:\Users\JBarr\anaconda3\envs\myenv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'split'

In [12]:

#4 clusters 
premType_4_Clusters = KMeans(n_clusters=4,random_state=0).fit(premType_scaled)



c:\Users\JBarr\anaconda3\envs\myenv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:

#calculate the Silhouette score for each cluster 
#this score ranges from âˆ’1 to +1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.

# premType_2_Clusters_silhouette_avg = silhouette_score(premType_scaled, premType_2_Clusters.labels_)
# print('Silhouette score:', premType_2_Clusters_silhouette_avg )
# #score 0.930

# premType_4_Clusters_silhouette_avg = silhouette_score(premType_scaled, premType_4_Clusters.labels_)
# print('Silhouette score:', premType_4_Clusters_silhouette_avg )
# #score 0.944

#cluster membership
membership = pd.Series(premType_4_Clusters.labels_, index = moneyFlow.index)
moneyFlow['membership'] = membership
moneyFlow.head()
moneyFlow["membership"].unique()


#-------------------------Dataset Based on Clustering -------------------------#
#create four data sets according to membership 
moneyFlow_cluster_1 = moneyFlow[moneyFlow["membership"] == 0]
moneyFlow_cluster_2 = moneyFlow[moneyFlow["membership"] == 1]
moneyFlow_cluster_3 = moneyFlow[moneyFlow["membership"] == 2]
moneyFlow_cluster_4 = moneyFlow[moneyFlow["membership"] == 3]

#-------------------------Data Split-------------------------#
# Split the data into X and y for cluster 1
X_cluster_1 = moneyFlow_cluster_1.drop('Sentiment', axis=1)
y_cluster_1= moneyFlow_cluster_1['Sentiment']
# Split the data into training and testing sets for cluster 1
X_train_cluster_1, X_test_cluster_1, y_train_cluster_1, y_test_cluster_1 = train_test_split(X_cluster_1, y_cluster_1, test_size=0.3, random_state=42)

# Split the data into X and y for cluster 2
X_cluster_2 = moneyFlow_cluster_2.drop('Sentiment', axis=1)
y_cluster_2= moneyFlow_cluster_2['Sentiment']
# Split the data into training and testing sets for cluster 1
X_train_cluster_2, X_test_cluster_2, y_train_cluster_2, y_test_cluster_2 = train_test_split(X_cluster_2, y_cluster_2, test_size=0.3, random_state=42)

# Split the data into X and y for cluster 3
X_cluster_3 = moneyFlow_cluster_3.drop('Sentiment', axis=1)
y_cluster_3= moneyFlow_cluster_3['Sentiment']
# Split the data into training and testing sets for cluster 1
X_train_cluster_3, X_test_cluster_3, y_train_cluster_3, y_test_cluster_3 = train_test_split(X_cluster_3, y_cluster_3, test_size=0.3, random_state=42)

# Split the data into X and y for cluster 4
X_cluster_4 = moneyFlow_cluster_4.drop('Sentiment', axis=1)
y_cluster_4= moneyFlow_cluster_4['Sentiment']
# Split the data into training and testing sets for cluster 1
X_train_cluster_4, X_test_cluster_4, y_train_cluster_4, y_test_cluster_4 = train_test_split(X_cluster_4, y_cluster_4, test_size=0.3, random_state=42)

#-------------------------Feature Scaling-------------------------#
scaler = StandardScaler()
X_train_cluster_1 = scaler.fit_transform(X_train_cluster_1)
X_test_cluster_1 = scaler.fit_transform(X_test_cluster_1)
X_train_cluster_2 = scaler.fit_transform(X_train_cluster_2)
X_test_cluster_2= scaler.fit_transform(X_test_cluster_2)
X_train_cluster_3 = scaler.fit_transform(X_train_cluster_3)
X_test_cluster_3= scaler.fit_transform(X_test_cluster_3)
X_train_cluster_4 = scaler.fit_transform(X_train_cluster_4)
X_test_cluster_4= scaler.fit_transform(X_test_cluster_4)

#-------------------------Logisitc Regression Cluster 1-------------------------#
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train_cluster_1, y_train_cluster_1)
lr_pred_cluster_1 = lr.predict(X_test_cluster_1)

#-------------------------Logisitc Regression Classification Report and Confusion Matrix for Cluster 1-------------------------#
print("Logistic Regression classification report:")
print(classification_report(y_test_cluster_1, lr_pred_cluster_1))
print("Logistic Regression confusion matrix:")
print(confusion_matrix(y_test_cluster_1, lr_pred_cluster_1))


#-------------------------Random Forest Cluster 1-------------------------#
rf = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=42)
rf.fit(X_train_cluster_1, y_train_cluster_1)
rf_pred_cluster_1 = rf.predict(X_test_cluster_1)

#-------------------------Random Forest Classification Report and Confusion Matrix for Cluster 1-------------------------#
print("Random Forest classification report:")
print(classification_report(y_test_cluster_1, rf_pred_cluster_1))
print("Random Forest confusion matrix:")
print(confusion_matrix(y_test_cluster_1, rf_pred_cluster_1))


#-------------------------Neural Netowrk-------------------------#
nn = MLPClassifier(hidden_layer_sizes=(64,),max_iter=1000, random_state=42)
nn.fit(X_train_cluster_1, y_train_cluster_1)
nn_pred_cluster_1= nn.predict(X_test_cluster_1)


#-------------------------Neural Network Classification Report and Confusion Matrix-------------------------#
print("Neural Network classification report:")
print(classification_report(y_test_cluster_1, nn_pred_cluster_1))
print("Neural Network confusion matrix:")
print(confusion_matrix(y_test_cluster_1, nn_pred_cluster_1))



#-------------------------KNN-------------------------#
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_cluster_1, y_train_cluster_1)
knn_pred_cluster_1 = knn.predict(X_test_cluster_1)


#-------------------------KNN Classification Report and Confusion Matrix-------------------------#
print("KNN Confusion Matrix:")
print(confusion_matrix(y_test_cluster_1, knn_pred_cluster_1))
print("\nKNN Classification Report:")
print(classification_report(y_test_cluster_1, knn_pred_cluster_1))


